# Übungen zur Textklassifikation mit Naive Bayes

### Aufgabe 1: Theorie
Lesen Sie [Sebastian Raschkas Artikel](http://sebastianraschka.com/Articles/2014_naive_bayes_1.html) über das Naive Bayes Verfahren und beantworten Sie folgende Fragen:
1. Weshalb ist *zusätzliche Glättung* hilfreich?

Ausprägungen unserer Features, die im Trainingsdatensatz nicht vorkommen, führen dazu, dass die **klassenbedingte Wahrscheinlichkeit** hierfür bei 0 liegt. Mit der Konsequenz, dass auch die posterior Wahrscheinlichkeit hierfür stets bei 0 liegen wird. Um dieses Problem zu umgehen, wird ein zusätzlicher Parameter $\alpha$ im Bayes-Modell addiert. 

2. Was versteht man unter dem Begriff *Stop Word*?

Ein "Stop Word" ist ein Wort, welches für Erkennung des Textinhaltes/Klassifikation keinen Informationsgewinn bedeutet. Typischerweise sind dies Artikel, Konjunktionen etc. Möglichkeiten zum Ausfiltern dieser Wörter vor der Modellbildung sind der Abgleich gegen eine "Stop-Word" Wörterbuch der betrachteten Sprache(n). Möglich ist auch aus den Testdaten eine geordnete Liste mit den Häufigkeiten der verwendeten Wörter zu erstellen. Hieraus kann man nun händisch eine Liste mit Stop-Words erstellen und diese aus meinen Testdaten herausnehmen.

3. Warum eignet sich das *Multi-variate Bernoulli Naive Bayes Verfahren* für die Spam-Erkennung?

Für eine allgemeine Textklassifikation ist das Multi-variate-Bernoulli Naive Bayes Verfahren theoretisch dem Multinomialen Modell unterlegen, da hier die Häufigkeit eines Feautures nicht berücksichtigt wird. Allerdings ist für die Güte eines Modells vielmehr die Auswahl des Vokabulars (Herausnehmen von Stop-Words etc.) entscheidend.



### Aufgabe 2: Anwendung auf Spam-Erkennung
Vwerwenden Sie den den [Spambase Datensatz](https://archive.ics.uci.edu/ml/datasets/Spambase), um ein Modell zu generieren, mit Hilfe dessen Sie Mails als [Spam](https://de.wikipedia.org/wiki/Spam#Begriffsherkunft) oder [Ham](http://bjc.berkeley.edu/bjc-r/cur/programming/data/spam-ham/1-introduction.html) einstufen können. Testen Sie Ihr Modell mit Hilfe eines kleinen [Spam-Ham Datensatzes](http://bjc.berkeley.edu/bjc-r/cur/programming/data/spam-ham/spam-ham-500.txt) aus Berkeley.

## Bemerkung
Es ist schon interessant zu sehen, welchen Einfluss die [Monty Python Gruppe](https://de.wikipedia.org/wiki/Monty_Python) auf die Informationsverarbeitung hat. Wie gerade (hoffentlich) gelesen, ist deren [Spam-Sketch](https://de.wikipedia.org/wiki/Spam-Sketch) maßgeblich für die Verwendung des Begriffs bei E-Mails. Was aber viele nicht wissen ist, dass auch der Name der [Programmiersprache Python](https://de.wikipedia.org/wiki/Python_(Programmiersprache)#Entwicklungsgeschichte) auf die Komiker zurück geht.

--> Aufbereitung der spambase.data Datei in openoffice calc und notepad++:
1. Import in Calc (Komma als Trenner)
2. Öffnen der spambase.names Datei - Löschen der Überflüssigen Informationen (Zeile 1-33)
3. Ersetzen von ':' durch ','
4. Leerzeichen und Zeilenumbrüche löschen (ersetzen durch '')
5. Anhängen des Attributes 'spam'
6. Kopieren der Attributnamen in die erste Zeile der spambase.data Datei

In [589]:
import pandas as pd
import sklearn
import sklearn.model_selection as ms
import sklearn.feature_extraction.text as text
import sklearn.naive_bayes as nb
import numpy as np

Modifizierte spambase-Daten-Datei einlesen:

In [590]:
sb = pd.read_csv("data/spambase.csv")

In [591]:
sb.head()

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total,spam
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1


Definition der Klassifikationen (labels)  $\mathbf{y}$:

In [502]:
y = sb['spam']

Definieren der *Merkmalsmatrix* $\mathbf{X}$. Entspricht der Eingelesenen Matrix sb bis auf die Klassifikation selbst ('spam') und die Feature <br>
'capital_run_length_average', <br>
'capital_run_length_longest', <br>
'capital_run_length_total'.<br>
Diese werden der Einfachheit halber (im Hinblick auf die Analyse des spam-ham-500 Datensatzes) herausgenommen. Somit besteht die Merkmalsmatrix unserem Dataframe sb ohne die letzten vier Spalten.


In [503]:
X = sb[sb.columns[:54]]

In [601]:
X.head()

,char_freq_!,char_freq_#,char_freq_$,char_freq_(,char_freq_;,char_freq_[,word_freq_000,word_freq_1999,word_freq_3d,word_freq_415,...,word_freq_re,word_freq_receive,word_freq_remove,word_freq_report,word_freq_table,word_freq_technology,word_freq_telnet,word_freq_will,word_freq_you,word_freq_your
0,0.778,0.000,0.000,0.000,0.00,0.0,0.00,0.00,0.0,0.0,...,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.64,1.93,0.96
1,0.372,0.048,0.180,0.132,0.00,0.0,0.43,0.07,0.0,0.0,...,0.00,0.21,0.21,0.21,0.0,0.0,0.0,0.79,3.47,1.59
2,0.276,0.010,0.184,0.143,0.01,0.0,1.16,0.00,0.0,0.0,...,0.06,0.38,0.19,0.00,0.0,0.0,0.0,0.45,1.36,0.51
3,0.137,0.000,0.000,0.137,0.00,0.0,0.00,0.00,0.0,0.0,...,0.00,0.31,0.31,0.00,0.0,0.0,0.0,0.31,3.18,0.31
4,0.135,0.000,0.000,0.135,0.00,0.0,0.00,0.00,0.0,0.0,...,0.00,0.31,0.31,0.00,0.0,0.0,0.0,0.31,3.18,0.31


Sortieren des Vokabulars in der Merkmalsmatrix für ein einfacheres Anwenden des Modells auf den Spam-Ham Datensatzes später. 

In [645]:
X.sort(axis=1,inplace=True)

C:\Users\Alex\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: sort(....) is deprecated, use sort_index(.....)
  if __name__ == '__main__':


In [646]:
(X_train, X_test, y_train, y_test) = ms.train_test_split(X, y, test_size=.2)

In [647]:
bnb = ms.GridSearchCV(nb.BernoulliNB(), param_grid={'alpha':np.logspace(-2., 2., 50)})
bnb.fit(X_train, y_train);

In [648]:
bnb.score(X_test, y_test)

0.88165038002171547

Überprüfen, welchen Einfluss der Wegfall der "capital run length"-Feature hatte:

In [649]:
X_all = sb[sb.columns[:57]]
(X_train, X_test, y_train, y_test) = ms.train_test_split(X_all, y, test_size=.2)
bnb_all = ms.GridSearchCV(nb.BernoulliNB(), param_grid={'alpha':np.logspace(-2., 2., 50)})
bnb_all.fit(X_train, y_train)
bnb_all.score(X_test, y_test)

0.89467969598262753

--> Der Wegfall dieser Feature hat anscheinend zumindest bei den Ausgewählten Testdaten keinen großen Effekt.

Einlesen des Spam-Ham-Datensatzes zum Testen:

In [607]:
sh = pd.read_csv("data/spam-ham-500.txt",sep="\t",usecols=[0,1],names=("spam","message"))

In [608]:
sh.head()

,spam,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


Überprüfen, ob die Klassifizierung korrekt eingelesen wurde:

In [609]:
sh[(sh['spam'] != 'ham') & (sh['spam'] != 'spam')]

,spam,message
45,ham nice car!,NaN
496,ham OK.,NaN


In den Datensätzen 45 und 496 scheint das Trennzeichen zu fehlen. Manuelle Korrektur:

In [610]:
sh['message'][45] = sh['spam'][45].partition(' ')[2]
sh['spam'][45] = sh['spam'][45].partition(' ')[0]
sh['message'][496] = sh['spam'][496].partition(' ')[2]
sh['spam'][496] = sh['spam'][496].partition(' ')[0]

Überprüfen, ob Korrektur erfolgreich war:

In [611]:
sh.iloc[(45,496),]

,spam,message
45,ham,nice car!
496,ham,OK.


Ersetzen der Klassifizierungsausprägungen:
ham = 0;
spam = 1

In [612]:
sh["spam"].replace({'ham' : 0, 'spam': 1 },inplace=True)

Überprüfen, ob es bei der Ersetzung zu fehlern kam:

In [613]:
sh[(sh['spam'] != 0) & (sh['spam'] != 1)]

,spam,message


**Erstellen der Merkmalsmatrix für den neuen Datensatz:<br>**
Extrahieren der Feature Wörter/Zeichen:

In [614]:
features = pd.DataFrame.from_dict(X.keys().tolist())

In [638]:
features.head()

,0
0,!
1,#
2,$
3,(
4,;


Für die Erstellung des Vokabulars die ersten 10 Zeichen (word\_freq\_ bzw. char\_freq\_) löschen:

In [625]:
i = 0
for i in range(0,54) :
    features[0][i] = features[0][i][10:]

Merkmalsmatrix erstellen:

In [656]:
tf = text.TfidfVectorizer(token_pattern=r"(?u)\b\w+\b|!|\;|\(|\[|\!|\$|\#")
X_sh = tf.fit(features[0][:54])
X_sh = tf.transform(sh['message'])

Trefferquote des Klassifikators testen:

In [661]:
bnb.score(X_sh, sh['spam'])

0.85799999999999998

--> Scheinbar gute Anpassung des Modells. **ABER:**

In [662]:
print(bnb.predict(X_sh))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

Es wird einfach so gut wie nichts als Spam erkannt. --> **Modell** für den Spam_Ham_500 Datensatz **unbrauchbar**. Das Vokabular für unser Modell war vermutlich zu sehr eingeschränkt.

So sah wohl eine typische Spam-Nachricht des spambase Datensatzes aus:

In [659]:
print(bnb.predict(tf.transform([
    "Give us your EMail-Address and you will receive all our products for free"
    ])))

[1]


Hier hingegen eine Spam-Nachricht aus dem Spam-Ham Datensatz:

In [660]:
print(bnb.predict(tf.transform([
    "Thanks for your subscription to Ringtone UK your mobile will be charged Â£5/month Please confirm by replying YES or NO. If you reply NO you will not be charged"
    ])))

[0]
